In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import warnings

warnings.filterwarnings("ignore", category=RuntimeWarning, module="sklearn")

In [9]:
df = pd.read_csv(r"/Users/loganheydt/Documents/GitHub/Kaggle_competitions/Wine_Quality/data/train.csv")

In [10]:
df_train, df_test = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df["quality"]
)

In [11]:
def apply_thresholds(scores, thresholds, min_class):
    return (np.digitize(scores, thresholds) + min_class).astype(int)

def optimize_thresholds(scores, y_true, classes, n_iter=30):
    """
    Coordinate-descent threshold optimizer to maximize QWK.
    thresholds length = len(classes)-1
    """
    classes = list(classes)
    min_c, max_c = classes[0], classes[-1]
    k = len(classes)

    qs = np.linspace(0, 1, k+1)[1:-1]
    thr = np.quantile(scores, qs)

    def qwk_for(thr_):
        pred = apply_thresholds(scores, thr_, min_c)
        pred = np.clip(pred, min_c, max_c)
        return cohen_kappa_score(y_true, pred, weights="quadratic")

    best = qwk_for(thr)

    for _ in range(n_iter):
        improved = False
        for i in range(len(thr)):
            cand = np.quantile(scores, np.linspace(0.05, 0.95, 41))
            lo = thr[i-1] if i > 0 else -np.inf
            hi = thr[i+1] if i < len(thr)-1 else np.inf
            cand = cand[(cand > lo) & (cand < hi)]
            if len(cand) == 0:
                continue

            local_best_thr = thr[i]
            local_best = best

            for v in cand:
                thr_try = thr.copy()
                thr_try[i] = v
                val = qwk_for(thr_try)
                if val > local_best:
                    local_best = val
                    local_best_thr = v

            if local_best > best:
                thr[i] = local_best_thr
                best = local_best
                improved = True

        if not improved:
            break

    return thr, best

def make_monotone_constraints(X, monotone_dict):
    """
    X: feature DF
    monotone_dict: e.g. {"alcohol": 1} (1 = increasing, -1 = decreasing, 0 = none)
    Returns constraints string for xgboost: "(0,1,0,...)"
    """
    cons = []
    for c in X.columns:
        cons.append(int(monotone_dict.get(c, 0)))
    return "(" + ",".join(map(str, cons)) + ")"

In [12]:
def stacked_oof_threshold_cv(
    df,
    target_col="quality",
    drop_cols=None,
    n_splits=5,
    seed=42,
    monotone_feature="alcohol"
):
    if drop_cols is None:
        drop_cols = []
    if "id" in df.columns and "id" not in drop_cols:
        drop_cols = drop_cols + ["id"]

    y = df[target_col].astype(int).values
    X = df.drop(columns=[target_col] + drop_cols, errors="ignore").replace([np.inf, -np.inf], np.nan)

    classes = sorted(np.unique(y).tolist())
    min_c, max_c = classes[0], classes[-1]

    # --- monotone constraints (only alcohol increasing by default) ---
    mono = {}
    if monotone_feature in X.columns:
        mono[monotone_feature] = 1
    monotone_constraints = make_monotone_constraints(X, mono)

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)

    # Store OOF predictions for each base model
    oof_xgb_mono = np.zeros(len(X), dtype=float)
    oof_rf = np.zeros(len(X), dtype=float)
    oof_ridge = np.zeros(len(X), dtype=float)

    fold_scores = []

    for fold, (tr, va) in enumerate(kf.split(X), start=1):
        X_tr, X_va = X.iloc[tr], X.iloc[va]
        y_tr, y_va = y[tr], y[va]

        # --- Base 1: XGBRegressor with monotone alcohol ---
        xgb_mono = XGBRegressor(
            n_estimators=900,
            learning_rate=0.05,
            max_depth=4,
            subsample=0.8,
            colsample_bytree=0.8,
            reg_lambda=1.0,
            random_state=seed,
            n_jobs=-1,
            monotone_constraints=monotone_constraints,
        )
        xgb_mono.fit(X_tr, y_tr)
        oof_xgb_mono[va] = xgb_mono.predict(X_va)

        # --- Base 2: RandomForest ---
        rf = RandomForestRegressor(
            n_estimators=600,
            random_state=seed,
            n_jobs=-1,
            min_samples_leaf=2
        )
        rf.fit(X_tr, y_tr)
        oof_rf[va] = rf.predict(X_va)

        # --- Base 3: Ridge on standardized-ish inputs (Ridge can help as a “linear anchor”) ---
        # Ridge needs numeric matrix; NaNs -> fill (simple median impute)
        X_tr_r = X_tr.copy()
        X_va_r = X_va.copy()
        med = X_tr_r.median(numeric_only=True)
        X_tr_r = X_tr_r.fillna(med)
        X_va_r = X_va_r.fillna(med)

        ridge = Ridge(alpha=1.0, random_state=seed)
        ridge.fit(X_tr_r, y_tr)
        oof_ridge[va] = ridge.predict(X_va_r)

        # quick fold check (optional)
        fold_rmse = np.sqrt(mean_squared_error(y_va, oof_xgb_mono[va]))
        fold_scores.append({"fold": fold, "xgb_mono_rmse": fold_rmse, "n_val": len(va)})

    fold_df = pd.DataFrame(fold_scores)

    # ---- STACKING: meta-model trained on OOF predictions ----
    oof_stack_X = np.vstack([oof_xgb_mono, oof_rf, oof_ridge]).T

    meta = Ridge(alpha=1.0, random_state=seed)
    meta.fit(oof_stack_X, y)
    oof_meta_scores = meta.predict(oof_stack_X)  # latent continuous score

    # ---- 1️⃣ Joint thresholds across folds: optimize on OOF predictions ----
    thr, best_oof_qwk = optimize_thresholds(oof_meta_scores, y, classes=classes, n_iter=40)

    # Evaluate with those single global thresholds
    preds = apply_thresholds(oof_meta_scores, thr, min_c)
    preds = np.clip(preds, min_c, max_c)
    qwk = cohen_kappa_score(y, preds, weights="quadratic")

    summary = {
        "n_splits": n_splits,
        "classes": classes,
        "monotone_constraints_used": monotone_constraints,
        "OOF_QWK_global_thresholds": float(qwk),
        "OOF_QWK_during_optimization": float(best_oof_qwk),
        "thresholds": thr.tolist(),
    }

    return summary, fold_df, pd.DataFrame({
        "y_true": y,
        "oof_xgb_mono": oof_xgb_mono,
        "oof_rf": oof_rf,
        "oof_ridge": oof_ridge,
        "oof_meta_score": oof_meta_scores,
        "oof_pred_class": preds
    })

In [13]:
summary, fold_df, oof_df = stacked_oof_threshold_cv(
    df_train,
    target_col="quality",
    drop_cols=["id"],
    n_splits=5
)

thresholds = summary["thresholds"]

In [14]:
import numpy as np
import pandas as pd

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor


def make_monotone_constraints(X: pd.DataFrame, monotone_dict: dict) -> str:
    """
    X: feature DF (column order matters)
    monotone_dict: e.g. {"alcohol": 1} (1=increasing, -1=decreasing, 0=none)
    Returns an xgboost constraint string like "(0,1,0,...)"
    """
    cons = [int(monotone_dict.get(c, 0)) for c in X.columns]
    return "(" + ",".join(map(str, cons)) + ")"


def apply_thresholds(scores: np.ndarray, thresholds, min_class: int) -> np.ndarray:
    thresholds = np.asarray(thresholds, dtype=float)
    return (np.digitize(scores, thresholds) + min_class).astype(int)


def fit_final_stacked_model(
    df: pd.DataFrame,
    target_col: str = "quality",
    drop_cols=None,
    seed: int = 42,
    monotone_feature: str = "alcohol",
    thresholds=None,
    xgb_params: dict = None,
    rf_params: dict = None,
    meta_params: dict = None,
):
    """
    Fits base models + meta model on ALL provided data (typically df_train).
    Uses provided thresholds (recommended: thresholds learned from OOF CV on df_train).

    Returns a model_bundle dict to be used with predict_final().
    """
    if drop_cols is None:
        drop_cols = []
    if "id" in df.columns and "id" not in drop_cols:
        drop_cols = drop_cols + ["id"]

    y = df[target_col].astype(int).values
    X = df.drop(columns=[target_col] + drop_cols, errors="ignore").copy()
    X = X.replace([np.inf, -np.inf], np.nan)

    classes = sorted(np.unique(y).tolist())
    min_c, max_c = classes[0], classes[-1]

    # --- monotone constraints (alcohol increasing by default) ---
    mono = {}
    if monotone_feature in X.columns:
        mono[monotone_feature] = 1
    monotone_constraints = make_monotone_constraints(X, mono)

    # --- default params (you can override) ---
    if xgb_params is None:
        xgb_params = dict(
            n_estimators=900,
            learning_rate=0.05,
            max_depth=4,
            subsample=0.8,
            colsample_bytree=0.8,
            reg_lambda=1.0,
            random_state=seed,
            n_jobs=-1,
        )
    if rf_params is None:
        rf_params = dict(
            n_estimators=600,
            random_state=seed,
            n_jobs=-1,
            min_samples_leaf=2
        )
    if meta_params is None:
        meta_params = dict(alpha=1.0, random_state=seed)

    # --- Base 1: XGBRegressor with monotone constraints ---
    xgb_mono = XGBRegressor(
        **xgb_params,
        monotone_constraints=monotone_constraints,
    )
    xgb_mono.fit(X, y)
    pred_xgb = xgb_mono.predict(X)

    # --- Base 2: RandomForest ---
    rf = RandomForestRegressor(**rf_params)
    rf.fit(X, y)
    pred_rf = rf.predict(X)

    # --- Base 3: Ridge (needs imputation for NaNs) ---
    medians = X.median(numeric_only=True)
    X_r = X.fillna(medians)
    ridge = Ridge(alpha=1.0, random_state=seed)
    ridge.fit(X_r, y)
    pred_ridge = ridge.predict(X_r)

    # --- Meta model trained on base predictions ---
    stack_train = np.vstack([pred_xgb, pred_rf, pred_ridge]).T
    meta = Ridge(**meta_params)
    meta.fit(stack_train, y)

    # If thresholds not provided, we default to quantiles (OK for quick use, but prefer OOF-learned thresholds)
    if thresholds is None:
        qs = np.linspace(0, 1, len(classes)+1)[1:-1]
        thresholds = np.quantile(meta.predict(stack_train), qs)

    model_bundle = {
        "classes": classes,
        "min_c": min_c,
        "max_c": max_c,
        "thresholds": np.asarray(thresholds, dtype=float),

        "feature_cols": X.columns.tolist(),
        "medians_for_ridge": medians,

        "monotone_constraints_used": monotone_constraints,

        "xgb_mono": xgb_mono,
        "rf": rf,
        "ridge": ridge,
        "meta": meta,
    }
    return model_bundle


def predict_final(model_bundle: dict, df_new: pd.DataFrame):
    """
    Predicts:
      - pred_class: integer labels in [min_c, max_c]
      - scores: continuous latent score from meta model
    """
    X_new = df_new[model_bundle["feature_cols"]].copy()
    X_new = X_new.replace([np.inf, -np.inf], np.nan)

    p1 = model_bundle["xgb_mono"].predict(X_new)
    p2 = model_bundle["rf"].predict(X_new)

    Xr = X_new.fillna(model_bundle["medians_for_ridge"])
    p3 = model_bundle["ridge"].predict(Xr)

    stack = np.vstack([p1, p2, p3]).T
    scores = model_bundle["meta"].predict(stack)

    pred_class = apply_thresholds(scores, model_bundle["thresholds"], model_bundle["min_c"])
    pred_class = np.clip(pred_class, model_bundle["min_c"], model_bundle["max_c"]).astype(int)

    return pred_class, scores

In [15]:
df_train, df_test = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df["quality"]
)

# 1) Run your OOF stacking + threshold optimization on df_train to get thresholds
summary, fold_df, oof_df = stacked_oof_threshold_cv(df_train, target_col="quality", drop_cols=["id"])
thr = summary["thresholds"]

# 2) Fit final model on df_train using those thresholds
final_model = fit_final_stacked_model(df_train, target_col="quality", drop_cols=["id"], thresholds=thr)

# 3) Predict on df_test
pred_test, score_test = predict_final(final_model, df_test)

qwk_test = cohen_kappa_score(df_test["quality"], pred_test, weights="quadratic")
print("Test QWK:", qwk_test)

Test QWK: 0.2602667625160814
